In [ ]:
# to automatically reload modules who's content has changed
%load_ext autoreload
%autoreload 2
# configure matplotlib
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
from IPython.core.debugger import set_trace

In [ ]:
import time
import numpy as np
import scipy
import sklearn.gaussian_process as sk_gp
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed") # annoying seaborn bug
import seaborn as sns;sns.set()

In [ ]:
import sys
sys.path.append('..')
import distributed_gp as dgp

In [ ]:
noise = 0.5
def to_fit(x):
    v = np.sin(x*2) * 0.2*x**2 + 4*np.cos(x)
    if noise != 0:
        v += np.random.normal(loc=0, scale=noise, size=np.asarray(x).shape)
    return v

In [ ]:
np.random.seed(0)
num_samples = 1000
X = np.random.uniform(0, 10, size=(num_samples, 1))
y = to_fit(X)

xs = np.linspace(0, 10, num=200)

In [ ]:
def plot_to_fit(ax=None):
    if ax is None:
        fig, ax = plt.subplots(figsize=(20, 8))
    ax.plot(xs, to_fit(xs), '--', color='grey')
    ax.scatter(X, y, color='C3', zorder=5)
def plot_prediction(mu_pred, sig_pred, ax=None):
    if ax is None:
        fig, ax = plt.subplots(figsize=(20, 8))
    ax.plot(xs, mu_pred)
    ax.fill_between(xs, (mu_pred-sig_pred).flatten(), (mu_pred+sig_pred).flatten(), alpha=0.2)
plot_to_fit()

In [ ]:
def full_GP():
    np.random.seed(0) 
    start = time.perf_counter()
    training_iterations = 1
    k = sk_gp.kernels.ConstantKernel(5) * sk_gp.kernels.RBF() + sk_gp.kernels.WhiteKernel()
    gp = sk_gp.GaussianProcessRegressor(kernel=k, alpha=0, n_restarts_optimizer=training_iterations-1)
    gp.fit(X, y)
    print('fitted in {}'.format(time.perf_counter()-start))
    print('optimised theta =', np.exp(gp.kernel_.theta))
    print('likelihood =', gp.log_marginal_likelihood())
    
    mu_pred, sig_pred = gp.predict(xs.reshape(-1, 1), return_std=True)
    sig_pred = sig_pred.reshape(-1, 1)
    
    fig, ax = plt.subplots(figsize=(20, 8))
    plot_to_fit(ax)
    plot_prediction(mu_pred, sig_pred, ax=ax)
    
full_GP()

In [ ]:
def dist_GP():
    np.random.seed(0) 
    start = time.perf_counter()
    k = sk_gp.kernels.ConstantKernel() * sk_gp.kernels.RBF() + sk_gp.kernels.WhiteKernel()
    gp = dgp.DistributedGP(X, y, num_experts=10, kernel=k)
    gp.optimise_params(iterations=1)
    print('fitted in {}'.format(time.perf_counter()-start))
    print('optimised theta =', np.exp(gp.kernel.theta))
    print('likelihood =', gp.log_marginal_likelihood())
    
    mu, var = gp.predict_POE(xs.reshape(-1, 1))
    sigma = np.sqrt(var)
    
    fig, ax = plt.subplots(figsize=(20, 8))
    plot_to_fit(ax)
    plot_prediction(mu, sigma, ax=ax)
    #for e in gp.experts: plot_prediction(*e.predict(xs.reshape(-1,1)), ax=ax)
    
dist_GP()